<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class4-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 4.1**

**Announcements**
* For next class (Mon 10/17), watch Video No. 7: *The Convolution Theorem*
* Lab 4 is a bit more intensive than Lab 3.
* Reminder: Next week's lab (No. 5) is the Midterm Lab (20% of final grade)




# Optional stuff

## Interactive Matplotlib

Install `ipympl` for  Matplotlib


In [ ]:
#!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()
# %matplotlib widget

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
As always, start by importing the "usual" modules we'll be using...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal

rc('animation', html='jshtml')

path = '/content/drive/My Drive/eces435-work/class4.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'tr505'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Helper functions

## `myPlot()`
A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

## `myPlotFFT()`
Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, y_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim < 0:
      plt.ylim(y_lim, 0)
    elif y_lim > 0:
      plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  #return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

# Load this week's audio sample
*New clip for Week 4!*

* 44.1 kHz sampling rate

In [ ]:
fc44,fs44 = sf.read(path + 'FinalCountdown-44kHz.wav')
ipd.Audio(fc44,rate=fs44)

In [ ]:
myPlot(fc44)

# Quantization

### `quantize()` (from last Class & Lab)

Quantize an input signal to a specified precision per sample (in bits).
* Required arguments:
  * `sig` Input signal (first argument)
  * `n_bits` The number of bits per sample
* Optional arguments:
  * `norm=True/False` Normalize the quantized signal and noise outputs to be within $\pm1$ (default: `True`)
* Outputs:
  * `sig_q` The quantized signal
  * `q_noise` The quantization noise

In [ ]:
def quantize(sig, n_bits, norm=True):
  q_scale = 2**(n_bits-1)

  sig_q = np.floor(sig * q_scale)
  q_noise = sig * q_scale - sig_q

  if norm:
    sig_q = sig_q / q_scale
    q_noise = q_noise / q_scale

  return sig_q, q_noise

### Quantize sample to 4 bits

In [ ]:
n_bits = 4

fc44_q, q_noise = quantize(fc44, n_bits, norm=False)
ipd.Audio(fc44_q, rate=fs44)

In [ ]:
myPlot(fc44_q, x_lim=[12,12.1], y_lim=[-3,3])

## What does quantization do in the frequency domain?

### Time to write a `mySpectrogram()` function
A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 
  S_dBFS = 
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

### Spectrogram of original clip

In [ ]:
mySpectrogram(fc44)

### Spectrogram of quantized clip

In [ ]:
fig1 = mySpectrogram(fc44_q)
fig1

### Save figure to Google Drive

After you're satisfied with the spectrogram figure.

In [ ]:
fig1.savefig(path+'spectrogram/'+username+'fc_q'+str(n_bits)+'.png')

# Bitrate

What's the bitrate for these various quantizations?
* $b_r = f_s \cdot B \cdot C$ in bits / second
  * $f_s$ is the sampling rate
  * $B$ is bits per sample
  * $C$ is the number of channels

In [ ]:
n_bits = 16

br44_16 = fs44 * n_bits
print(br44_16)

## What are other bitrates?

Compute some variations
* Different sampling rates
* Different bit depths (quantization levels)

*And what is a typical streaming bitrate (Spotify, Apple Music, etc.)?*

In [ ]:
my_br = 

# Sample Rate Conversion: The easy way

* What if we need to change the sampling rate of a signal?

## Create 11025 Hz sample rate version

* The easy way (downsampling)


In [ ]:
fs11 = fs44 / 4
fc11 =                      # Downsample by factor of 4
ipd.Audio(fc11,rate=fs11)

Check the spectrogram of your downsampled signal

In [ ]:
mySpectrogram(fc11, fs11, y_lim=[0,22050])

## Upsampling

* Insert zeros between existing samples

In [ ]:
L = 4
up4 = np.zeros( len(fc11)*L - (L-1) )
up4[::4] = fc11                # Put our 11 kHz wave every 4th sample

ipd.Audio(up4, rate=fs44)

In [ ]:
myPlot(up4[:500])

In [ ]:
mySpectrogram(up4)

## Smoothing: Moving average

In [ ]:
L = 9  # Number of samples to average

N = len(up4)
s_ma = np.zeros(N)               # Create an output array of zeros

for n in range(N - L):
  s_ma[n] = np.sum(up4[n:n+L]) / L   # Compute moving average of L samples

In [ ]:
myPlot(s_ma[:500])

In [ ]:
ipd.Audio(s_ma, rate=fs44)

In [ ]:
h_ma = np.ones(L) / L

myPlotFFT(4*h_ma, n_fft=2048, win='rect')

In [ ]:
myPlotFFT(s_ma)

In [ ]:
mySpectrogram(s_ma)

## Smoothing: Weighted moving average

In [ ]:
L = 9  # Number of samples to average
h_tri = np.bartlett(L)

s_tri = np.zeros(N)               # Create an output array of zeros

for n in range(N - L):
  s_tri[n] = np.sum( up4[n:n+L] * h_tri )   # Compute weighted average of L samples

In [ ]:
plt.plot(h_tri, '.')

In [ ]:
myPlot(s_tri[:500])

In [ ]:
ipd.Audio(s_tri, rate=fs44)

In [ ]:
myPlotFFT(h_tri, n_fft=2048, win='rect', y_lim=-100)

In [ ]:
mySpectrogram(s_tri)

In [ ]:
myPlotFFT(s_tri, win='rect')

## Write your own smoothing function, `smoothie()`

Inputs:
* `x` signal to be smoothed
* `h` smoothing function (window)

Output:
* `s` smoothed signal

In [ ]:
def smoothie(x, h):
  N = len(x)
  L = len(h)

  s = np.zeros(N)                   # Create an output array of zeros
  for n in range(N - L):
    s[n] = 

  return s

#### Test out your function

In [ ]:
L = 17  # Number of samples to average
h_hann = np.hanning(L)

s_hann = smoothie(up4, h_hann)

In [ ]:
fig2 = myPlot(s_hann[:1500])
fig2

In [ ]:
fig2.savefig(path + 'smoothing/' + username + '-hann.png')

In [ ]:
ipd.Audio(s_hann, rate=fs44)

In [ ]:
myPlotFFT(h_hann, n_fft=2048, win='rect', y_lim=-100)

In [ ]:
myPlotFFT(s_hann, win='rect')

In [ ]:
mySpectrogram(s_hann)

# Sampling Rate Conversion: Strategy

To convert from one sampling rate, $M$, to a different sampling rate $L$, here's a strategy:
* Upsample by $L$
* Filter (smooth) with a cutoff of $1/L$ or $1/M$, whichever is smaller.
* Downsample by $M$

## Convert from 11kHz to 16kHz

### Upsample by 16

In [ ]:
L = 16
up16 = np.zeros( len(fc11)*L - (L-1) )
up16[::L] = fc11                # Put our 11 kHz wave every Lth sample
myPlot(up16[:500*L])

### Smoothing (filtering)

* The lowpass cutuff should be either $1/M$ or $1/L$ in normalized frequency (where 1.0 is the Nyquist frequency), whichever is smaller (depends if you're going to a higher rate or a lower rate).

* In this case (going from 11kHz to 16 kHz) the cutoff is approximately 1/16 (in normalized frequency).

In [ ]:
1/L

In [ ]:
N_win =
hann = np.hanning(N_win)

myPlotFFT(hann, fs=2, win='rect', n_fft=2048, y_lim=-100)

In [ ]:
up16_s = smoothie(up16, hann)

myPlot(up16_s[:500*L])

In [ ]:
myPlotFFT(up16, fs=2)

In [ ]:
myPlotFFT(up16_s, fs=2)

### Downsample by 11

In [ ]:
fc16 = up16_s[::11]
myPlot(fc16[:500])

In [ ]:
ipd.Audio(fc16, rate=16000)

In [ ]:
myPlotFFT(fc16, fs=16000)

In [ ]:
mySpectrogram(fc16, fs=16000)